In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import ne_chunk
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from bs4 import BeautifulSoup 
import re
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,roc_auc_score 
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
stop_words=set(nltk.corpus.stopwords.words('english'))
import keras
from keras.preprocessing.text import one_hot,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Flatten ,Embedding,Input,LSTM, SimpleRNN
from keras.models import Model
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec

In [ ]:
rev_frame=pd.read_csv('Reviews.csv')
df=rev_frame.copy()

df = df[:2000]

In [ ]:
df=df[['Text','Score']]

In [ ]:
df['review']=df['Text']
df['rating']=df['Score']
df.drop(['Text','Score'],axis=1,inplace=True)

In [ ]:
df.drop_duplicates(subset=['rating','review'],keep='first',inplace=True) 

In [ ]:
def mark_sentiment(rating):
    if(rating<=3):
        return 0
    else:
        return 1

In [ ]:
df['sentiment']=df['rating'].apply(mark_sentiment)

In [ ]:
df.drop(['rating'],axis=1,inplace=True)

In [ ]:
df['sentiment'].value_counts()

In [ ]:
pos_df=df.loc[df.sentiment==1,:][:50000]
neg_df=df.loc[df.sentiment==0,:][:50000]

In [ ]:
#combining
df=pd.concat([pos_df,neg_df],ignore_index=True)

In [ ]:
# shuffling rows
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape) 
df.head()

In [ ]:
import gensim
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens

df['tokens'] = df['review'].apply(preprocess_text)
w2v_model = gensim.models.Word2Vec(sentences=df['tokens'], vector_size=300, window=10, min_count=1)

In [ ]:
w2v_model.train(df['tokens'], total_examples=len(df['tokens']), epochs=10)

In [ ]:
vocab = list(w2v_model.wv.key_to_index.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v_model.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict))

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['review'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['review'])

In [ ]:
max_rev_len=1565
vocab_size = len(tok.word_index)
embed_dim=300

In [ ]:
pad_rev= pad_sequences(encd_rev, maxlen=max_rev_len, padding='post')

In [ ]:
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
  embed_vector=word_vec_dict.get(word)
  if embed_vector is not None:
    embed_matrix[i]=embed_vector

In [ ]:
Y=keras.utils.to_categorical(df['sentiment'])
x_train,x_test,y_train,y_test=train_test_split(pad_rev,Y,test_size=0.20,random_state=42)

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(SimpleRNN(32))
model.add(Dense(2, activation='sigmoid'))
model.summary()

In [ ]:
from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
model=Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_rev_len,embeddings_initializer=Constant(embed_matrix)))
model.add(LSTM(32))
model.add(Dense(2, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-3),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=1,batch_size=64,validation_data=(x_test,y_test))